# How to Get the Source Documents used to produce a RAG answer

* [Link to the Documentation Page.](https://python.langchain.com/docs/use_cases/question_answering/sources)

## Goal
* Show user the sources that were used to generate the RAG answer returning the Documents that were retrieved in each generation.

## Dependencies and Necessary Modules
* Same as with the project we develop for the Quickstart Guide (see notebook 159-rag-quickstart).

## .env file
* OpenAI API key.
* LangSmith credentials.
* Our LangSmith project name: RAGgetSources

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Open LangSmith to track the following operations
* smith.langchain.com

## The initial RAG App without sources

In [1]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
# Load, chunk and index the contents of the blog.
bs_strainer = bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs_strainer},
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [5]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps agents plan and execute tasks more efficiently by dividing them into manageable subtasks. Task decomposition can be achieved through various methods, including using prompting techniques, task-specific instructions, or human inputs.'

## Adding sources

In [7]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

response = rag_chain_with_source.invoke("What is Task Decomposition")

In [8]:
print(f"Question: {response['question']}")

Question: What is Task Decomposition


In [9]:
print(f"Answer: {response['answer']}")

Answer: Task decomposition is a technique used to break down complex tasks into smaller and simpler steps, making it easier for agents to plan and execute them. It involves transforming big tasks into manageable ones by thinking step by step or exploring multiple reasoning possibilities at each step. This process can be done using prompting techniques, task-specific instructions, or human inputs.


In [10]:
# Iterate over the context list and print each document's content and metadata
for index, document in enumerate(response['context']):
    print(f"\nDocument {index + 1} Content:\n{document.page_content}")
    print(f"Document {index + 1} Metadata:\n{document.metadata}")



Document 1 Content:
Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.
Document 1 Metadata:
{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

Document 2 Content:
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be B

## Aditional techniques to extract Citations in RAG Apps
* [Link to the Documentation Page](https://python.langchain.com/docs/use_cases/question_answering/citations).